In [7]:
import mediapipe as mp
import cv2
import numpy as np
import uuid
import os

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
mp_drawing_styles = mp.solutions.drawing_styles

In [8]:
def decide_gesture(inout_list):
    
    if all([not(x ^ y)for x, y in zip(inout_list,[True, False, True, True, True])]):
        return 'one'
    elif all([not(x ^ y)for x, y in zip(inout_list,[True, False, False, True, True])]):
        return 'two'
    elif all([not(x ^ y)for x, y in zip(inout_list,[True, False, False, False, True])]):
        return 'three'
    elif all([not(x ^ y)for x, y in zip(inout_list,[True, False, False, False, False])]):
        return 'four'
    elif all([not(x ^ y)for x, y in zip(inout_list,[False, False, False, False, False])]):
        return 'five'
    elif all([not(x ^ y)for x, y in zip(inout_list,[False, True, True, True, False])]):
        return 'six'
    elif all([not(x ^ y)for x, y in zip(inout_list,[False, False, True, True, True])]):
        return 'seven'
    elif all([not(x ^ y)for x, y in zip(inout_list,[False, False, False, True, True])]):
        return 'eight'
    elif all([not(x ^ y)for x, y in zip(inout_list,[True, False, True, True, False])]):
        return 'rock1'
    elif all([not(x ^ y)for x, y in zip(inout_list,[False, False, True, True, False])]):
        return 'rock2'

In [31]:
def formula_big(x1, x2, x3, y1, y2, y3):
    
    mul_x = 640
    mul_y = 480
    x1 = x1 * mul_x
    x2 = x2 * mul_x
    x3 = x3 * mul_x
    y1 = y1 * mul_y
    y2 = y2 * mul_y
    y3 = y3 * mul_y
    
    a = (y1 - y2) / (x1 - x2)
    b = y1 - a * x1
    
    #cv2.line(image, (int(round((100 - b) / a)), 100), (int(round((480 - b) / a)), 640), (0, 0, 255), 1)
    
    if x1 > x2:
        if x3 * a + b > y3 - abs(x1 - x2 + y1 - y2):
            return True
        return False
    else:
        if x3 * a + b > y3:
            return False
        return True

In [27]:
def formula_other(x1, x2, x3, y1, y2, y3):
    
    mul_x = 640
    mul_y = 480
    x1 = x1 * mul_x
    x2 = x2 * mul_x
    x3 = x3 * mul_x
    y1 = y1 * mul_y
    y2 = y2 * mul_y
    y3 = y3 * mul_y
    
    a = (y1 - y2) / (x1 - x2)
    b = y1 - a * x1
    
    #cv2.line(image, (int(round((100 - b) / a)), 100), (int(round((480 - b) / a)), 640), (0, 0, 255), 1)
    
    
    if x1 > x2:
        if x3 * a + b > y3:
            return True
        return False
    else:
        if x3 * a + b > y3 + 30:
            return False
        return True
            

<img src=https://i.imgur.com/qpRACer.png />

In [35]:
import math

def draw_and_get_finger_angles2(image, results):
    
    pos = 20
    RLmultiply = 0
    distance = None
    gesture = None
    inout = ''
    # Loop through hands
    for RL, hand in enumerate(results.multi_hand_landmarks):
        inout_list = []
        try:
            if results.multi_handedness[RL].classification[0].label == 'Right':
                RLmultiply = 1
            else:
                RLmultiply = 0
        except:
            pass
        a = np.array([hand.landmark[4].x, hand.landmark[4].y]) #大拇指尖
        b = np.array([hand.landmark[8].x, hand.landmark[8].y]) #食指尖
        c = np.array([hand.landmark[12].x, hand.landmark[12].y]) #中指尖
        d = np.array([hand.landmark[16].x, hand.landmark[16].y]) #無名指尖
        e = np.array([hand.landmark[20].x, hand.landmark[20].y]) #小指尖
        f = np.array([hand.landmark[5].x, hand.landmark[5].y]) #食指根
        g = np.array([hand.landmark[9].x, hand.landmark[9].y]) #中指根
        h = np.array([hand.landmark[13].x, hand.landmark[13].y]) #無名指根
        i = np.array([hand.landmark[17].x, hand.landmark[17].y]) #小指根
        j = np.array([hand.landmark[0].x, hand.landmark[0].y]) #手腕
        
        #大拇指
        inout_list.append(formula_big(f[0], j[0], a[0], f[1], j[1], a[1]))
        #食指
        inout_list.append(not formula_other(f[0], g[0], b[0], f[1], g[1], b[1]))
        #中指
        inout_list.append(not formula_other(f[0], g[0], c[0], f[1], g[1], c[1]))
        #無名指
        inout_list.append(not formula_other(f[0], g[0], d[0], f[1], g[1], d[1]))
        #小拇指
        inout_list.append(not formula_other(f[0], g[0], e[0], f[1], g[1], e[1]))
        
        
        cv2.putText(image, " ".join(str(x) for x in inout_list), (pos + num * 80 + RLmultiply * 320, 20),
               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

        if RLmultiply == 1:
            distance = math.sqrt((hand.landmark[4].x - hand.landmark[12].x)**2 + (hand.landmark[4].y - hand.landmark[12].y)**2)
            cv2.putText(image, str(round(distance, 2)), (int(hand.landmark[0].x * 640), int(hand.landmark[0].y * 480) + 30),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, inout, (int(hand.landmark[0].x * 640), int(hand.landmark[0].y * 480) + 60),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            
        if RLmultiply == 0:
            gesture = decide_gesture(inout_list)
            cv2.putText(image, gesture, (int(hand.landmark[0].x * 640), int(hand.landmark[0].y * 480) + 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, inout, (int(hand.landmark[0].x * 640), int(hand.landmark[0].y * 480) + 60),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
    return image, distance, gesture

In [37]:
cap = cv2.VideoCapture(0)
counter = 0 # 計算偵數，讓音樂間隔 N 偵以上播放
N = 0
threshold = 0.2 
Status = True

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands: 
    while cap.isOpened():
        ret, frame = cap.read()
        
        
        # BGR 2 RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Flip on horizontal
        image = cv2.flip(image, 1)
        
        # Set flag
        image.flags.writeable = False
        
        # Detections
        results = hands.process(image)
        
        # Set flag to true
        image.flags.writeable = True
        
        # RGB 2 BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Rendering results
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                                        mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                        mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2),
                                         )
        
            # Draw angles to image from joint list
            image, distance, gesture = draw_and_get_finger_angles2(image, results)
            #print("distance:", distance)
            #print("gesture:", gesture)
            
        # Save our image
        #cv2.imwrite(os.path.join('Output Images', '{}.jpg'.format(uuid.uuid1())), image)
        
        try:
            if distance > threshold:
                Status = True
            if gesture and distance < threshold and counter > N and Status: # counter>20 讓音樂間隔 N 偵以上播放
                play_chord(num2chord[gesture])
                counter = 0
                Status = False
        except:
            pass
        
        cv2.imshow('Hand Tracking', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
        counter = counter +1 

cap.release()
cv2.destroyAllWindows()

Thread D
Thread C
Thread C
Thread C
Thread C
Thread C


In [5]:
num2chord = {
    'one':"C",
    'two':"A",
    'three':"D",
    'four':"G"
}

In [7]:
# num2chord = {
#     'one':"gcd1_chord",
#     'two':"gcd2_chord",
#     'three':"gcd3_chord",
#     'four':"gcd4_chord"
# }

In [3]:
!pip install playsound

  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7026 sha256=92ddc61bc5a60723077c3c63607dfda88e5bcc856408a5025bd5a032005f0216
  Stored in directory: c:\users\cvc016\appdata\local\pip\cache\wheels\73\cd\cf\9750b618d54bd81c20e4c34fb24a423a5b095920367cdb3f71
Successfully built playsound


In [36]:
from playsound import playsound
import threading
import time

def init_play_chord(chord):
    print("Thread", chord)
    playsound('music/{}_chord.mp3'.format(chord)) # 讓初次載入時發出聲音
    
def play_chord(chord):
    threads[chords[chord]] = threading.Thread(target = init_play_chord, args = (chord,))
    threads[chords[chord]].start()

chords = {
    "C": 0,
    "A": 1,
    "D": 2,
    "G": 3,
#     "gcd1_chord": 0,
#     "gcd2_chord": 1,
#     "gcd3_chord": 2,
#     "gcd4_chord": 3
}

# 建立 4 個子執行緒
threads = []
for i, chord_name in enumerate(chords):
    threads.append(threading.Thread(target = init_play_chord, args = (chord_name,)))
    threads[i].start()
    time.sleep(2)
    
for i in range(4):
    threads[i].join()

Thread C
Thread A
Thread D
Thread G


In [2]:
joint_list2 = [[8, 7, 6, 5], [12, 11, 10, 9], [16, 15, 14, 13], [20, 19, 18, 17]]

In [ ]:
import math

def draw_and_get_finger_angles2(image, results, joint_list):
    
    pos = 20
    RLmultiply = 0
    distance = None
    gesture = None
    inout = ''
    # Loop through hands
    for RL, hand in enumerate(results.multi_hand_landmarks):
        angle_list = []
        try:
            if results.multi_handedness[RL].classification[0].label == 'Right':
                RLmultiply = 1
            else:
                RLmultiply = 0
        except:
            pass
        e = np.array([hand.landmark[0].x, hand.landmark[0].y]) # Fifth coord
        h = np.array([hand.landmark[4].x, hand.landmark[4].y])
        g = np.array([hand.landmark[5].x, hand.landmark[5].y])
        
        
        inout_list.append(formula(e[0], g[0], h[0], e[1], g[1], h[1]))
        
        #Loop through joint sets
        for num, joint in enumerate(joint_list2):
            a = np.array([hand.landmark[joint[0]].x, hand.landmark[joint[0]].y]) # First coord
            b = np.array([hand.landmark[joint[1]].x, hand.landmark[joint[1]].y]) # Second coord
            c = np.array([hand.landmark[joint[2]].x, hand.landmark[joint[2]].y]) # Third coord
            d = np.array([hand.landmark[joint[3]].x, hand.landmark[joint[3]].y]) # Fuorth coord
            
            radians1 = np.arctan2(b[1] - c[1], b[0]-c[0]) - np.arctan2(d[1]-c[1], d[0]-c[0])
            radians2 = np.arctan2(c[1] - d[1], c[0]-d[0]) - np.arctan2(e[1]-d[1], e[0]-d[0])
            angle1 = np.abs(radians1*180.0/np.pi)
            angle2 = np.abs(radians2*180.0/np.pi)
            
            
            if angle1 > 180.0:
                angle1 = 360-angle1
            if angle2 > 180.0:
                angle2 = 360-angle2
            
            angle_list.append((angle1, angle2))
                
            cv2.putText(image, str(round(angle1, 2)), (pos + num * 80 + RLmultiply * 320, 20),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, str(round(angle2, 2)), (pos + num * 80 + RLmultiply * 320, 40),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
        if RLmultiply == 1:
            distance = math.sqrt((hand.landmark[4].x - hand.landmark[12].x)**2 + (hand.landmark[4].y - hand.landmark[12].y)**2)
            cv2.putText(image, str(round(distance, 2)), (int(hand.landmark[0].x * 640), int(hand.landmark[0].y * 480) + 30),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, inout, (int(hand.landmark[0].x * 640), int(hand.landmark[0].y * 480) + 60),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            
        if RLmultiply == 0:
            gesture = decide_gesture(angle_list)
            cv2.putText(image, gesture, (int(hand.landmark[0].x * 640), int(hand.landmark[0].y * 480) + 30),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, inout, (int(hand.landmark[0].x * 640), int(hand.landmark[0].y * 480) + 60),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
    return image, distance, gesture